In [ ]:
!pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
!pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
from snowflake.snowpark import Session

In [ ]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [ ]:
session = get_session()
# session.close()
session

In [ ]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [1]:
from snowflake.ml.registry import model_registry

ModuleNotFoundError: No module named 'snowflake.ml'

In [ ]:
!pip freeze | grep -i cloud

In [ ]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
registry = model_registry.ModelRegistry(session=session, database_name="test_database_prak", schema_name="test_schema_prak")

In [ ]:
registry.get_history().show()

In [ ]:
mv = registry.log_model(model_obj,
                   model_name="my_model",
                   version_name="v1",
                   conda_dependencies=["scikit-learn"],
                   comment="My awesome ML model",
#                    metrics={"score": 96},
                   sample_input_data=train_features)